# Projeto: Detecção de Queimadas com Machine Learning
Este projeto tem como objetivo detectar áreas queimadas utilizando aprendizado de máquina com dados geoespaciais e espectrais.


In [2]:
!pip install numpy matplotlib pandas torch scikit-learn rasterio spyndex

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 12.8/12.8 MB 68.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 8.1/8.1 MB 84.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 11.0/11.0 MB 86.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/241.3 MB ? eta -:--:--
   --- ----------------------------------- 23.6/241.3 MB 117.2 MB/s eta 0:00:02
   ------- ------------------------------- 48.2/241.3 MB 116.4 MB/s eta 0:00:02
   ----------- --------------------------- 71.6/241.3 MB 116.2 MB/s eta 0:00:02
   --------------- ----------------------- 95.4/241.3 MB 116.7 MB/s eta 0:00:02
   ------------------ ------------------- 119.8/241.3 MB 116.4 MB/s eta 0:00:02
   ----


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Bibliotecas de manipulação e visualização
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Machine Learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Dados geoespaciais
import rasterio
from rasterio import plot
import spyndex


In [ ]:
raster = rasterio.open("imagem_satellite.tif")
plot.show(raster)

In [ ]:
features = df[['NDVI', 'NBR', 'BURNED_AREA']]
labels = df['queimada']  # 0 ou 1

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, labels, test_size=0.2, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

In [ ]:
y_pred_proba = clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

fpr, tpr, _ = roc_curve(y_test, y_pred_proba)

plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.grid(True)
plt.show()